### **Project: expressEats**

***Part-4: In this Notebook we modelled the problem as sequence generation(receipe generation) which is a bonus!(We will explore in depth this approach in Deep Learning Course)***


Look at our codebase below for more details

### **Load Dataset**

In [ ]:
import pandas as pd

# Loading the dataset to examine its structure
file_path = '/kaggle/input/6000-indian-food-recipes-dataset/IndianFoodDatasetCSV.csv'
data = pd.read_csv(file_path)

# Displaying the first few rows of the dataset
data.head()


,Srno,RecipeName,TranslatedRecipeName,Ingredients,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions,TranslatedInstructions,URL
0,1,Masala Karela Recipe,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...","6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...","To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...
1,2,टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...,Spicy Tomato Rice (Recipe),"2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बी...","2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,टमाटर पुलियोगरे बनाने के लिए सबसे पहले टमाटर क...,"To make tomato puliogere, first cut the tomato...",http://www.archanaskitchen.com/spicy-tomato-ri...
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...","1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...","To begin making the Ragi Vermicelli Recipe, fi...",http://www.archanaskitchen.com/ragi-vermicelli...
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...","500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,To begin making Gongura Chicken Curry Recipe f...,http://www.archanaskitchen.com/gongura-chicken...
4,5,आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 बड़ा चमच्च चना दाल,1 बड़ा चमच्च सफ़ेद उरद दाल,2...","1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,आंध्रा स्टाइल आलम पचड़ी बनाने के लिए सबसे पहले ...,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...


### ***Data Cleaning***

In [ ]:
# Remove duplicates
data_cleaned = data.drop_duplicates(subset=['TranslatedIngredients', 'TranslatedInstructions'])

# Remove rows with any null values in the two main columns
data_cleaned = data_cleaned.dropna(subset=['TranslatedIngredients', 'TranslatedInstructions'])

# Checking the shape of the cleaned data
data_cleaned.shape


(6864, 15)

In [ ]:
import re

def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

# Apply cleaning function to both columns
data_cleaned['TranslatedIngredients'] = data_cleaned['TranslatedIngredients'].apply(clean_text)
data_cleaned['TranslatedInstructions'] = data_cleaned['TranslatedInstructions'].apply(clean_text)

# Displaying the first few rows of the cleaned data
data_cleaned.head()


,Srno,RecipeName,TranslatedRecipeName,Ingredients,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions,TranslatedInstructions,URL
0,1,Masala Karela Recipe,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",6 karela bitter gourd pavakkai deseededsalt ...,15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",to begin making the masala karela recipedeseed...,https://www.archanaskitchen.com/masala-karela-...
1,2,टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...,Spicy Tomato Rice (Recipe),"2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बी...",21 2 cups rice cooked 3 tomatoes 3 teaspoons...,5,10,15,3,South Indian Recipes,Main Course,Vegetarian,टमाटर पुलियोगरे बनाने के लिए सबसे पहले टमाटर क...,to make tomato puliogere first cut the tomatoe...,http://www.archanaskitchen.com/spicy-tomato-ri...
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",112 cups rice vermicelli noodles thin1 onion ...,20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",to begin making the ragi vermicelli recipe fir...,http://www.archanaskitchen.com/ragi-vermicelli...
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",500 grams chicken2 onion chopped1 tomato cho...,15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,to begin making gongura chicken curry recipe f...,http://www.archanaskitchen.com/gongura-chicken...
4,5,आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 बड़ा चमच्च चना दाल,1 बड़ा चमच्च सफ़ेद उरद दाल,2...",1 tablespoon chana dal 1 tablespoon white urad...,10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,आंध्रा स्टाइल आलम पचड़ी बनाने के लिए सबसे पहले ...,to make andhra style alam pachadi first heat o...,https://www.archanaskitchen.com/andhra-style-a...


### ***Dealing with missing values and train test splitting***

In [ ]:
# Remove duplicates
data = data.drop_duplicates(subset=['TranslatedIngredients', 'TranslatedInstructions'])

# Remove rows with any null values in the two main columns
data = data.dropna(subset=['TranslatedIngredients', 'TranslatedInstructions'])

# Checking the shape of the data after preprocessing
data.shape


(6864, 15)

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Checking the shape of the training and testing data
train_data.shape, test_data.shape


((5491, 15), (1373, 15))

### ***Tokenising Data***

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fitting the vectorizer to the training data
vectorizer.fit(train_data['TranslatedIngredients'])

# Transforming the ingredients to bag-of-words representation
X_train_bow = vectorizer.transform(train_data['TranslatedIngredients'])
X_test_bow = vectorizer.transform(test_data['TranslatedIngredients'])

# Checking the shape of the transformed data
X_train_bow.shape, X_test_bow.shape


((5491, 2646), (1373, 2646))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the tokenizer
tokenizer = Tokenizer()

# Fitting the tokenizer on the ingredients and instructions
tokenizer.fit_on_texts(data['TranslatedIngredients'])
tokenizer.fit_on_texts(data['TranslatedInstructions'])

# Tokenizing the data
X_train_seq = tokenizer.texts_to_sequences(train_data['TranslatedIngredients'])
X_test_seq = tokenizer.texts_to_sequences(test_data['TranslatedIngredients'])
y_train_seq = tokenizer.texts_to_sequences(train_data['TranslatedInstructions'])
y_test_seq = tokenizer.texts_to_sequences(test_data['TranslatedInstructions'])

# Padding the sequences to ensure uniform length
max_length = 100  # This is a parameter we might need to tune
X_train_seq = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq = pad_sequences(X_test_seq, maxlen=max_length, padding='post')
y_train_seq = pad_sequences(y_train_seq, maxlen=max_length, padding='post')
y_test_seq = pad_sequences(y_test_seq, maxlen=max_length, padding='post')


### ***Model Training***

***Serious Note: Accuracy does not make sense as a metric here as we are modelling sequence generation***

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a file path pattern for saving the model
model_checkpoint = ModelCheckpoint(
    filepath="model_weights_epoch_{epoch:02d}.h5",  # Change the file path as needed
    save_weights_only=True,  # Save only the model's weights
    save_best_only=True,  # Save weights at every epoch, not just the best
    monitor="val_loss",  # You can choose a metric to monitor, e.g., validation loss
    mode="auto",  # Automatically choose the best direction to minimize the monitored metric
    verbose=1  # Display progress during training
)

# Define model parameters
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
embedding_dim = 100  # Size of the word embeddings
lstm_units = 128  # Number of units in LSTM layer

# Build the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=64, validation_data=(X_test_seq, y_test_seq), callbacks=[model_checkpoint])


Epoch 1/50
86/86 [==============================] - ETA: 0s - loss: 7.4480 - accuracy: 0.0540
Epoch 1: val_loss improved from inf to 6.53174, saving model to model_weights_epoch_01.h5
86/86 [==============================] - 290s 3s/step - loss: 7.4480 - accuracy: 0.0540 - val_loss: 6.5317 - val_accuracy: 0.0610
Epoch 2/50
86/86 [==============================] - ETA: 0s - loss: 6.4950 - accuracy: 0.0609
Epoch 2: val_loss improved from 6.53174 to 6.51694, saving model to model_weights_epoch_02.h5
86/86 [==============================] - 283s 3s/step - loss: 6.4950 - accuracy: 0.0609 - val_loss: 6.5169 - val_accuracy: 0.0610
Epoch 3/50
86/86 [==============================] - ETA: 0s - loss: 6.4741 - accuracy: 0.0609
Epoch 3: val_loss improved from 6.51694 to 6.51506, saving model to model_weights_epoch_03.h5
86/86 [==============================] - 280s 3s/step - loss: 6.4741 - accuracy: 0.0609 - val_loss: 6.5151 - val_accuracy: 0.0610
Epoch 4/50
86/86 [==============================] 